In [15]:
import wandb
wandb.login(key='fac5cd7f576f5d6d2591b0e77385c09a7922b210')
wandb.init(
    # set the wandb project where this run will be logged
    project="Part_5_1",
    
    # track hyperparameters and run metadata
    config={
    "learning_rate": 0.04,
    "architecture": "CNN",
    "dataset": "CIFAR-100",
    "epochs": 10,
    }
)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


epoch,▁█
val_accuracy1,▁▁
val_accuracy2,▁▁
epoch,1
val_accuracy1,0.0
val_accuracy2,0.0


In [16]:
import os
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

class CustomMultiMNISTDataset(Dataset):
    def __init__(self, root, transform=None):
        self.root = root
        self.transform = transform
        self.image_paths = []
        self.labels = []

        for folder in os.listdir(root):
            if os.path.isdir(os.path.join(root, folder)):
                digit1 = int(folder[0])
                digit2 = int(folder[1])
                if digit1 != digit2:
                    for filename in os.listdir(os.path.join(root, folder)):
                        self.image_paths.append(os.path.join(root, folder, filename))
                        self.labels.append((digit1, digit2))
        
    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, index):
        image_path = self.image_paths[index]
        label = self.labels[index]
        # print(image_path, label)
        with open(image_path, 'rb') as f:
            image = Image.open(f)
            if self.transform is not None:
                image = self.transform(image)
        
        return image, label

transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),  
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
])

train_dataset = CustomMultiMNISTDataset("../double_mnist_seed_123_image_size_64_64/train", transform=transform)
val_dataset = CustomMultiMNISTDataset("../double_mnist_seed_123_image_size_64_64/val", transform=transform)
test_dataset = CustomMultiMNISTDataset("../double_mnist_seed_123_image_size_64_64/test", transform=transform)

batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

print("Number of training samples:", len(train_dataset))
print("Number of validation samples:", len(val_dataset))
print("Number of testing samples:", len(test_dataset))

Number of training samples: 58000
Number of validation samples: 14000
Number of testing samples: 18000


In [17]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms

class MLPModel(nn.Module):
    def __init__(self, input_size, hidden_sizes, num_classes):
        super(MLPModel, self).__init__()
        layers = []
        layers.append(nn.Flatten())
        for i in range(len(hidden_sizes)):
            layers.append(nn.Linear(input_size if i == 0 else hidden_sizes[i - 1], hidden_sizes[i]))
            layers.append(nn.ReLU())
        layers.append(nn.Linear(hidden_sizes[-1], num_classes))
        layers.append(nn.Sigmoid())
        self.model = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.model(x)

In [18]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms

input_size = 64 * 64  
hidden_sizes = [128, 64]  
num_classes_per_digit = 10  
total_num_classes = 2 * num_classes_per_digit

model = MLPModel(input_size, [256, 128], total_num_classes)

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.005)

num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    for images, labels in train_loader:
        label1 = torch.zeros(len(labels[0]), num_classes_per_digit)
        label2 = torch.zeros(len(labels[1]), num_classes_per_digit)
        for i in range(len(labels[0])):
            label1[i, labels[0][i]] = 1
        for i in range(len(labels[1])):
            label2[i, labels[1][i]] = 1
        labels = torch.cat((label1, label2), dim=1)
        outputs = model(images)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    model.eval()
    with torch.no_grad():
        correct1 = 0
        correct2 = 0
        total = 0
        for images, labels in val_loader:
            label1 = torch.zeros(len(labels[0]), num_classes_per_digit)
            label2 = torch.zeros(len(labels[1]), num_classes_per_digit)
            for i in range(len(labels[0])):
                label1[i, labels[0][i]] = 1
            for i in range(len(labels[1])):
                label2[i, labels[1][i]] = 1
            labels = torch.cat((label1, label2), dim=1)
            outputs = model(images)
            predicted = (torch.sigmoid(outputs) > 0.5).float()
            total += labels.size(0)
            val1 =  (predicted[:, :num_classes_per_digit] == label1).all(1).sum().item()
            correct1 += val1
            val2 = (predicted[:, num_classes_per_digit:] == label2).all(1).sum().item()
            correct2 += val2

        accuracy1 = 100 * correct1 / total
        accuracy2 = 100 * correct2 / total
        print(f'Epoch {epoch+1}/{num_epochs}, Validation Accuracy (Digit 1): {accuracy1:.2f}%')
        print(f'Epoch {epoch+1}/{num_epochs}, Validation Accuracy (Digit 2): {accuracy2:.2f}%')
        # do wandb log of validation accuracy and epochs for both digits 1 and 2:
        wandb.log({"val_accuracy1": accuracy1, "val_accuracy2": accuracy2, "epoch": epoch})


model.eval()

## Evaluate on test set
with torch.no_grad():
    correct1 = 0
    correct2 = 0
    total = 0
    for images, labels in test_loader:
        label1 = torch.zeros(len(labels[0]), num_classes_per_digit)
        label2 = torch.zeros(len(labels[1]), num_classes_per_digit)
        for i in range(len(labels[0])):
            label1[i, labels[0][i]] = 1
        for i in range(len(labels[1])):
            label2[i, labels[1][i]] = 1
        labels = torch.cat((label1, label2), dim=1)
        outputs = model(images)
        predicted = (torch.sigmoid(outputs) > 0.5).float()
        total += labels.size(0)
        correct1 += (predicted[:, :num_classes_per_digit] == label1).all(1).sum().item()
        correct2 += (predicted[:, num_classes_per_digit:] == label2).all(1).sum().item()

    test_accuracy1 = 100 * correct1 / total
    test_accuracy2 = 100 * correct2 / total

print(f'Test Accuracy (Digit 1): {test_accuracy1:.2f}%')
print(f'Test Accuracy (Digit 2): {test_accuracy2:.2f}%')

Epoch 1/10, Validation Accuracy (Digit 1): 37.59%
Epoch 1/10, Validation Accuracy (Digit 2): 45.85%
Epoch 2/10, Validation Accuracy (Digit 1): 42.59%
Epoch 2/10, Validation Accuracy (Digit 2): 52.79%
Epoch 3/10, Validation Accuracy (Digit 1): 45.76%
Epoch 3/10, Validation Accuracy (Digit 2): 54.92%
Epoch 4/10, Validation Accuracy (Digit 1): 44.96%
Epoch 4/10, Validation Accuracy (Digit 2): 56.32%
Epoch 5/10, Validation Accuracy (Digit 1): 46.44%
Epoch 5/10, Validation Accuracy (Digit 2): 54.45%
Epoch 6/10, Validation Accuracy (Digit 1): 49.49%
Epoch 6/10, Validation Accuracy (Digit 2): 50.96%
Epoch 7/10, Validation Accuracy (Digit 1): 48.75%
Epoch 7/10, Validation Accuracy (Digit 2): 54.36%
Epoch 8/10, Validation Accuracy (Digit 1): 51.11%
Epoch 8/10, Validation Accuracy (Digit 2): 53.13%
Epoch 9/10, Validation Accuracy (Digit 1): 48.75%
Epoch 9/10, Validation Accuracy (Digit 2): 54.43%
Epoch 10/10, Validation Accuracy (Digit 1): 49.54%
Epoch 10/10, Validation Accuracy (Digit 2): 54.39

In [19]:
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
import copy

train_data_hyper = copy.deepcopy(train_dataset)
valid_data_hyper = copy.deepcopy(val_dataset)
test_data_hyper = copy.deepcopy(test_dataset)

df = pd.DataFrame({'image_path': train_dataset.image_paths, 'label': train_dataset.labels})
df = shuffle(df)

df = df.sample(frac=0.5)

train_data_hyper.image_paths = df.image_path.to_list()
train_data_hyper.labels = df.label.to_list()

df = pd.DataFrame({'image_path': val_dataset.image_paths, 'label': val_dataset.labels})
df = shuffle(df)
df = df.sample(frac=0.5)
valid_data_hyper.image_paths = df.image_path.to_list()
valid_data_hyper.labels = df.label.to_list()

df = pd.DataFrame({'image_path': test_dataset.image_paths, 'label': test_dataset.labels})
df = shuffle(df)
df = df.sample(frac=0.1)

test_data_hyper.image_paths = df.image_path.to_list()
test_data_hyper.labels = df.label.to_list()

print(len(train_data_hyper), len(valid_data_hyper), len(test_data_hyper))

batch_size = 64
train_loader_hyper = DataLoader(train_data_hyper, batch_size=batch_size, shuffle=True)
val_loader_hyper = DataLoader(valid_data_hyper, batch_size=batch_size)
test_loader_hyper = DataLoader(test_data_hyper, batch_size=batch_size)

29000 7000 1800


In [21]:
## Hyperparameter tuning with 4 tuples on validation dataset

hidden_layers = [[256, 256], [128, 128]]
learning_rates = [0.0025, 0.005]
num_epochs = 10
results = []

for hidden_layer in hidden_layers:
    for lr in learning_rates:

        model = MLPModel(input_size, hidden_layer, total_num_classes)
        criterion = nn.BCEWithLogitsLoss()
        optimizer = optim.Adam(model.parameters(), lr=lr)
        for epoch in range(num_epochs):
            model.train()
            for images, labels in train_loader_hyper:
                label1 = torch.zeros(len(labels[0]), num_classes_per_digit)
                label2 = torch.zeros(len(labels[1]), num_classes_per_digit)
                for i in range(len(labels[0])):
                    label1[i, labels[0][i]] = 1
                for i in range(len(labels[1])):
                    label2[i, labels[1][i]] = 1
                labels = torch.cat((label1, label2), dim=1)
                outputs = model(images)
                loss = criterion(outputs, labels)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
            
            model.eval()
            with torch.no_grad():
                correct1 = 0
                correct2 = 0
                total = 0
                for images, labels in val_loader_hyper:
                    label1 = torch.zeros(len(labels[0]), num_classes_per_digit)
                    label2 = torch.zeros(len(labels[1]), num_classes_per_digit)
                    for i in range(len(labels[0])):
                        label1[i, labels[0][i]] = 1
                    for i in range(len(labels[1])):
                        label2[i, labels[1][i]] = 1
                    labels = torch.cat((label1, label2), dim=1)
                    outputs = model(images)
                    predicted = (torch.sigmoid(outputs) > 0.5).float()
                    total += labels.size(0)
                    val1 =  (predicted[:, :num_classes_per_digit] == label1).all(1).sum().item()
                    correct1 += val1
                    val2 = (predicted[:, num_classes_per_digit:] == label2).all(1).sum().item()
                    correct2 += val2

                accuracy1 = 100 * correct1 / total
                accuracy2 = 100 * correct2 / total
                aggregate_accuracy = (accuracy1 + accuracy2) // 2
                print(f'Epoch {epoch+1}/{num_epochs}, Validation Accuracy (Digit 1): {accuracy1:.2f}%')
                print(f'Epoch {epoch+1}/{num_epochs}, Validation Accuracy (Digit 2): {accuracy2:.2f}%')

                if epoch == num_epochs - 1:
                    results.append((hidden_layer, lr, accuracy1, accuracy2, aggregate_accuracy))
                    wandb.log({"hidden_layer": hidden_layer, "lr": lr, "accuracy1": accuracy1, "accuracy2": accuracy2, "aggregate_accuracy": aggregate_accuracy})

            
        model.eval()

print(results)

Epoch 1/10, Validation Accuracy (Digit 1): 16.49%
Epoch 1/10, Validation Accuracy (Digit 2): 25.93%
Epoch 2/10, Validation Accuracy (Digit 1): 32.31%
Epoch 2/10, Validation Accuracy (Digit 2): 42.93%
Epoch 3/10, Validation Accuracy (Digit 1): 42.96%
Epoch 3/10, Validation Accuracy (Digit 2): 47.73%
Epoch 4/10, Validation Accuracy (Digit 1): 46.17%
Epoch 4/10, Validation Accuracy (Digit 2): 49.56%
Epoch 5/10, Validation Accuracy (Digit 1): 44.54%
Epoch 5/10, Validation Accuracy (Digit 2): 54.69%
Epoch 6/10, Validation Accuracy (Digit 1): 49.21%
Epoch 6/10, Validation Accuracy (Digit 2): 51.73%
Epoch 7/10, Validation Accuracy (Digit 1): 46.20%
Epoch 7/10, Validation Accuracy (Digit 2): 54.34%
Epoch 8/10, Validation Accuracy (Digit 1): 49.67%
Epoch 8/10, Validation Accuracy (Digit 2): 52.56%
Epoch 9/10, Validation Accuracy (Digit 1): 50.14%
Epoch 9/10, Validation Accuracy (Digit 2): 53.79%
Epoch 10/10, Validation Accuracy (Digit 1): 50.51%
Epoch 10/10, Validation Accuracy (Digit 2): 52.10

In [22]:
# sort results on basis of aggregate accuracy
results.sort(key=lambda x: x[4], reverse=True)

optimal_hidden_layer = results[0][0]
optimal_lr = results[0][1]

# evaluate test data on best model
model = MLPModel(input_size, optimal_hidden_layer, total_num_classes)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=optimal_lr)
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    for images, labels in train_loader:
        label1 = torch.zeros(len(labels[0]), num_classes_per_digit)
        label2 = torch.zeros(len(labels[1]), num_classes_per_digit)
        for i in range(len(labels[0])):
            label1[i, labels[0][i]] = 1
        for i in range(len(labels[1])):
            label2[i, labels[1][i]] = 1
        labels = torch.cat((label1, label2), dim=1)
        outputs = model(images)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    model.eval()

with torch.no_grad():
    correct1 = 0
    correct2 = 0
    total = 0
    for images, labels in test_loader_hyper:
        label1 = torch.zeros(len(labels[0]), num_classes_per_digit)
        label2 = torch.zeros(len(labels[1]), num_classes_per_digit)
        for i in range(len(labels[0])):
            label1[i, labels[0][i]] = 1
        for i in range(len(labels[1])):
            label2[i, labels[1][i]] = 1
        labels = torch.cat((label1, label2), dim=1)
        outputs = model(images)
        predicted = (torch.sigmoid(outputs) > 0.5).float()
        total += labels.size(0)
        correct1 += (predicted[:, :num_classes_per_digit] == label1).all(1).sum().item()
        correct2 += (predicted[:, num_classes_per_digit:] == label2).all(1).sum().item()
    test_accuracy1 = 100 * correct1 / total
    test_accuracy2 = 100 * correct2 / total

print("Test accuracy (digit 1):", test_accuracy1)
print("Test accuracy (digit 2):", test_accuracy2)

Test accuracy (digit 1): 53.27777777777778
Test accuracy (digit 2): 53.611111111111114


In [ ]:
import os
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms
from torch.utils.data import Dataset

class CustomMultiMNISTDataset(Dataset):
    def __init__(self, root, transform=None):
        self.root = root
        self.transform = transform
        self.image_paths = []
        self.labels = []

        for folder in os.listdir(root):
            if os.path.isdir(os.path.join(root, folder)):
                digit1 = int(folder[0])
                digit2 = int(folder[1])
                if digit1 != digit2:
                    for filename in os.listdir(os.path.join(root, folder)):
                        self.image_paths.append(os.path.join(root, folder, filename))
                        self.labels.append((digit1, digit2))

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, index):
        image_path = self.image_paths[index]
        label = self.labels[index]
        with open(image_path, 'rb') as f:
            image = Image.open(f)
            if self.transform is not None:
                image = self.transform(image)
        return image, label

class CNNModel(nn.Module):
    def __init__(self, num_classes):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(64 * 16 * 16, 128)
        self.fc2 = nn.Linear(128, num_classes)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = nn.functional.relu(self.conv1(x))
        x = nn.functional.max_pool2d(x, kernel_size=2, stride=2)
        x = nn.functional.relu(self.conv2(x))
        x = nn.functional.max_pool2d(x, kernel_size=2, stride=2)
        x = x.view(x.size(0), -1)
        x = nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
        return x
    
    def setKernelsize(self, kernelsize):
        self.kernelsize = kernelsize
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=kernelsize, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=kernelsize, stride=1, padding=1)

    def setStride(self, stride):
        self.stride = stride
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=self.kernelsize, stride=stride, padding=1)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=self.kernelsize, stride=stride, padding=1)

    def setDropout(self, dropout):
        self.dropout = nn.Dropout(dropout)

transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
])

train_dataset = CustomMultiMNISTDataset("../double_mnist_seed_123_image_size_64_64/train", transform=transform)
val_dataset = CustomMultiMNISTDataset("../double_mnist_seed_123_image_size_64_64/val", transform=transform)
test_dataset = CustomMultiMNISTDataset("../double_mnist_seed_123_image_size_64_64/test", transform=transform)

batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

print("Number of training samples:", len(train_dataset))
print("Number of validation samples:", len(val_dataset))
print("Number of testing samples:", len(test_dataset))

Number of training samples: 58000
Number of validation samples: 14000
Number of testing samples: 18000


In [ ]:
num_classes_per_digit = 10  
total_num_classes = 2 * num_classes_per_digit
model = CNNModel(total_num_classes)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0025)

num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    for images, labels in train_loader:
        label1 = torch.zeros(len(labels[0]), num_classes_per_digit)
        label2 = torch.zeros(len(labels[1]), num_classes_per_digit)
        for i in range(len(labels[0])):
            label1[i, labels[0][i]] = 1
        for i in range(len(labels[1])):
            label2[i, labels[1][i]] = 1
        labels = torch.cat((label1, label2), dim=1)
        outputs = model(images)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    model.eval()
    with torch.no_grad():
        correct1 = 0
        correct2 = 0
        total = 0
        for images, labels in val_loader:
            label1 = torch.zeros(len(labels[0]), num_classes_per_digit)
            label2 = torch.zeros(len(labels[1]), num_classes_per_digit)
            for i in range(len(labels[0])):
                label1[i, labels[0][i]] = 1
            for i in range(len(labels[1])):
                label2[i, labels[1][i]] = 1
            labels = torch.cat((label1, label2), dim=1)
            outputs = model(images)
            predicted = (torch.sigmoid(outputs) > 0.5).float()
            total += labels.size(0)
            correct1 += (predicted[:, :num_classes_per_digit] == label1).all(1).sum().item()
            correct2 += (predicted[:, num_classes_per_digit:] == label2).all(1).sum().item()

        accuracy1 = 100 * correct1 / total
        accuracy2 = 100 * correct2 / total
        print(f'Epoch {epoch + 1}/{num_epochs}, Validation Accuracy (Digit 1): {accuracy1:.2f}%')
        print(f'Epoch {epoch + 1}/{num_epochs}, Validation Accuracy (Digit 2): {accuracy2:.2f}%')
        wandb.log({"cnn_val_accuracy1": accuracy1, "cnn_val_accuracy2": accuracy2, "epoch": epoch})

Epoch 1/10, Validation Accuracy (Digit 1): 57.14%
Epoch 1/10, Validation Accuracy (Digit 2): 63.86%
Epoch 2/10, Validation Accuracy (Digit 1): 70.19%
Epoch 2/10, Validation Accuracy (Digit 2): 71.58%
Epoch 3/10, Validation Accuracy (Digit 1): 76.34%
Epoch 3/10, Validation Accuracy (Digit 2): 78.15%
Epoch 4/10, Validation Accuracy (Digit 1): 75.56%
Epoch 4/10, Validation Accuracy (Digit 2): 79.76%
Epoch 5/10, Validation Accuracy (Digit 1): 78.24%
Epoch 5/10, Validation Accuracy (Digit 2): 78.94%
Epoch 6/10, Validation Accuracy (Digit 1): 77.82%
Epoch 6/10, Validation Accuracy (Digit 2): 80.75%
Epoch 7/10, Validation Accuracy (Digit 1): 76.35%
Epoch 7/10, Validation Accuracy (Digit 2): 79.74%
Epoch 8/10, Validation Accuracy (Digit 1): 74.50%
Epoch 8/10, Validation Accuracy (Digit 2): 81.19%
Epoch 9/10, Validation Accuracy (Digit 1): 78.69%
Epoch 9/10, Validation Accuracy (Digit 2): 81.86%
Epoch 10/10, Validation Accuracy (Digit 1): 75.92%
Epoch 10/10, Validation Accuracy (Digit 2): 80.35

In [10]:
# Hyperparameter tuning on 1/10th training data, 1/10th validation data and 1/10th test data

learning_rates = [0.001, 0.005]
kernel_sizes = [3, 5]
stride_sizes = [1, 2]
dropout_rates = [0.2, 0.5]
num_classes_per_digit = 10
total_num_classes = 2 * num_classes_per_digit

results = []
hyperparams_list = [(0.001, 3, 1, 0.2), (0.005, 5, 2, 0.5), (0.001, 5, 1, 0.5), (0.005, 3, 2, 0.2)]

for hyperparams in hyperparams_list:
    lr = hyperparams[0]
    kernel_size = hyperparams[1]
    stride = hyperparams[2]
    dropout = hyperparams[3]
    model = CNNModel(total_num_classes)
    model.setKernelsize(kernel_size)
    model.setStride(stride)
    model.setDropout(dropout)
    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    num_epochs = 5
    for epoch in range(num_epochs):
        model.train()
        for images, labels in train_loader_hyper:
            label1 = torch.zeros(len(labels[0]), num_classes_per_digit)
            label2 = torch.zeros(len(labels[1]), num_classes_per_digit)
            for i in range(len(labels[0])):
                label1[i, labels[0][i]] = 1
            for i in range(len(labels[1])):
                label2[i, labels[1][i]] = 1
            labels = torch.cat((label1, label2), dim=1)
            outputs = model(images)
            loss = criterion(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        model.eval()
        with torch.no_grad():
            correct1 = 0
            correct2 = 0
            total = 0
            for images, labels in val_loader_hyper:
                label1 = torch.zeros(len(labels[0]), num_classes_per_digit)
                label2 = torch.zeros(len(labels[1]), num_classes_per_digit)
                for i in range(len(labels[0])):
                    label1[i, labels[0][i]] = 1
                for i in range(len(labels[1])):
                    label2[i, labels[1][i]] = 1
                labels = torch.cat((label1, label2), dim=1)
                outputs = model(images)
                predicted = (torch.sigmoid(outputs) > 0.5).float()
                total += labels.size(0)
                correct1 += (predicted[:, :num_classes_per_digit] == label1).all(1).sum().item()
                correct2 += (predicted[:, num_classes_per_digit:] == label2).all(1).sum().item()

            accuracy1 = 100 * correct1 / total
            accuracy2 = 100 * correct2 / total
            if epoch == num_epochs - 1:
                aggregate_accuracy = (accuracy1 + accuracy2) // 2

                print("For hyperparameters:", hyperparams)
                print("Aggregate accuracy:", aggregate_accuracy)
                results.append((lr, kernel_size, stride, dropout, accuracy1, accuracy2, aggregate_accuracy))
                wandb.log({"lr": lr, "kernel_size": kernel_size, "stride": stride, "dropout": dropout, "cnn_aggregate_accuracy": aggregate_accuracy})

For hyperparameters: (0.001, 3, 1, 0.2)
Aggregate accuracy: 68.0
For hyperparameters: (0.005, 5, 2, 0.5)
Aggregate accuracy: 67.0
For hyperparameters: (0.001, 5, 1, 0.5)
Aggregate accuracy: 70.0
For hyperparameters: (0.005, 3, 2, 0.2)
Aggregate accuracy: 69.0
